# 패키지 업데이트

In [1]:
# !pip install -U trl bitsandbytes -q

# 환경설정

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["WANDB_MODE"] = "disabled"

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

# 모델 불러오기

In [4]:
model_name = "KORMo-Team/KORMo-sft"

# 4bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델 로드 (4bit 양자화)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

print(f"✓ 모델 로드 완료: {model_name}")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✓ 모델 로드 완료: KORMo-Team/KORMo-sft


# 모델 추론

In [5]:
test_prompt = "영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?"

In [6]:
# Think 추론
print("[Think 추론]")
model.eval()
messages_think = [{"role": "user", "content": test_prompt}]
input_text_think = tokenizer.apply_chat_template(messages_think, tokenize=False, add_generation_prompt=True, enable_thinking=True)
inputs_think = tokenizer(input_text_think, return_tensors="pt").to(model.device)

with torch.inference_mode():
    outputs_think = model.generate(
        **inputs_think,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
response_think = tokenizer.decode(outputs_think[0], skip_special_tokens=True)
print(f"입력: {test_prompt}")
print(f"출력: {response_think}")

[Think 추론]
입력: 영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
출력: user
영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
assistant
<think>
영희가 처음에 가지고 있던 연필은 12개입니다.  
철수는 이 연필 중 절반을 가져갔으므로, 12개의 절반인 6개를 가져갔습니다.  
따라서 영희가 남은 연필은 12 - 6 = 6개입니다.  
영수는 공책 3개를 가져갔는데, 이는 연필과는 관련이 없으므로 무시해도 됩니다.  
최종적으로 영희에게 남은 연필의 개수는 6개입니다.
</think>

영희가 처음에 가지고 있던 연필은 12개입니다.  
철수는 이 중 절반을 가져갔으므로, $ \frac{12}{2} = 6 $개를 가져갔습니다.  
따라서 영희가 남은 연필의 수는 $ 12 - 6 = 6 $개입니다.  

**답: 6개**


In [7]:
# Non-think 추론
print("[Non-think 추론]")
messages = [{"role": "user", "content": test_prompt}]
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"입력: {test_prompt}")
print(f"출력: {response}")



[Non-think 추론]
입력: 영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
출력: user
영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
assistant
<think>

</think>
영희가 처음에 **연필 12개**를 가지고 있었습니다.
철수가 **절반**을 가져갔으므로:

\[
\frac{1}{2} \times 12 = 6 \text{개}
\]

따라서 철수가 6개를 가져가고, 영희에게 남은 연필은:

\[
12 - 6 = 6 \text{개}
\]

그 후, 영수가 **공책 3개**를 가져갔다고 했는데, 문제는 **연필의 개수**를 묻고 있습니다.  
공책은 연필과 관련이 없으므로, **영희의 연필 수에는 영향을 주지 않습니다**.

따라서, 영희에게 남은 연필의 개수는:

\[
\boxed{6}
\]


# KORMo SFT 데이터 일부 불러오기

In [8]:
dataset = load_dataset(
    'KORMo-Team/KORMo-tutorial-datasets',
    name='sft',
    split='train'
)

dataset = dataset.shuffle(seed=42).select(range(1000))
print(f"✓ 데이터셋 로드 완료: {len(dataset)}개 샘플")
print(f"✓ 데이터셋 컬럼: {dataset.column_names}")
print(f"\n샘플 예시:")
dataset[105]['conversation']

✓ 데이터셋 로드 완료: 1000개 샘플
✓ 데이터셋 컬럼: ['conversation']

샘플 예시:


[{'content': '세 친구가 호텔에서 아침 식사를 주문했는데, 각각은 견과류 롤 하나, 치즈 롤 하나, 과일 롤 하나가 들어 있는 가방을 기대하고 있었습니다. 아침 식사 준비 담당자는 총 9개의 롤을 포장하여 무작위로 각 친구에게 3개의 롤씩 담긴 가방을 나누어 주었습니다. 이때, 각각의 친구가 세 종류의 롤을 각각 하나씩 받았을 확률이 \\(\\frac{p}{q}\\)일 때, 여기서 \\(p\\)와 \\(q\\)는 서로소인 정수입니다. 이때의 \\(p+q\\) 값을 구하시오.',
  'reasoning_content': None,
  'role': 'user'},
 {'content': '우리는 세 친구에게 견과류 롤 3개, 치즈 롤 3개, 과일 롤 3개를 무작위로 3개씩 나누어 줄 때, **각 친구가 세 종류의 롤을 하나씩 받는 경우의 확률**을 구하고자 합니다. 이 확률이 $\\frac{p}{q}$ 꼴일 때, $p+q$ 값을 구하는 것이 목표입니다.\n\n---\n\n### 1. 전체 경우의 수\n\n총 9개의 롤을 3개씩 3명에게 나누어 주는 경우의 수를 구합니다.\n\n롤은 총 9개(견과류 3개, 치즈 3개, 과일 3개)이며, 이들을 3개씩 3명에게 나누어 줄 수 있는 경우의 수는 다음과 같이 계산됩니다:\n\n$$\n\\frac{1}{3!} \\cdot \\binom{9}{3} \\cdot \\binom{6}{3} \\cdot \\binom{3}{3}\n= \\frac{1}{6} \\cdot 84 \\cdot 20 = \\frac{1680}{6} = 280\n$$\n\n따라서 전체 가능한 배분 방법의 수는 **280가지**입니다.\n\n---\n\n### 2. 조건을 만족하는 경우의 수\n\n각 친구가 견과류, 치즈, 과일 롤을 각각 하나씩 받는 경우를 생각해봅니다. 이 경우는 다음과 같은 조건을 만족해야 합니다:\n\n- 각 친구는 견과류, 치즈, 과일 중 하나씩을 받음\n- 즉, 총 3개의 롤 종류가 각각 3개씩 있고, 각 친구에게 1개씩

In [9]:
# Chat template 적용 모습 확인

templated_text = tokenizer.apply_chat_template(
    conversation=dataset[105]['conversation'],
    tokenize=False,
    add_generation_prompt=False,
)
print(templated_text)

<|BOT|>user
세 친구가 호텔에서 아침 식사를 주문했는데, 각각은 견과류 롤 하나, 치즈 롤 하나, 과일 롤 하나가 들어 있는 가방을 기대하고 있었습니다. 아침 식사 준비 담당자는 총 9개의 롤을 포장하여 무작위로 각 친구에게 3개의 롤씩 담긴 가방을 나누어 주었습니다. 이때, 각각의 친구가 세 종류의 롤을 각각 하나씩 받았을 확률이 \(\frac{p}{q}\)일 때, 여기서 \(p\)와 \(q\)는 서로소인 정수입니다. 이때의 \(p+q\) 값을 구하시오.<|EOT|>
<|BOT|>assistant
<think>
문제를 이해해보면:
- 총 9개의 롤이 있습니다: 견과류 롤 3개, 치즈 롤 3개, 과일 롤 3개.
- 이 롤들을 무작위로 나누어 각 친구에게 3개씩 주게 됩니다.
- 관심 있는 사건: 각 친구가 세 종류의 롤을 하나씩 받는 경우.

이 사건이 일어날 확률을 $\frac{p}{q}$ 형태로 구하고, $p+q$를 구해야 합니다.

---

## 1. 전체 경우의 수

우선 총 9개의 롤을 3개씩 3묶음으로 나누는 경우의 수를 생각해보겠습니다.

- 9개의 롤 중 3개를 고르는 방법: $\binom{9}{3}$
- 남은 6개 중 3개를 고르는 방법: $\binom{6}{3}$
- 남은 3개는 자동으로 세 번째 친구에게 갑니다.

하지만 이 방법은 친구들을 구분하지 않는 경우를 고려하지 않으므로, 친구들을 구분하는 경우를 계산해야 합니다.

하지만, 여기서는 롤의 종류가 같아도 개별적인 롤로 구분하지 않고, 종류별로 3개씩 있다는 점을 고려해야 합니다. 따라서, 롤을 구분하지 않는 경우를 고려해야 합니다.

하지만, 이 문제는 "무작위로 각 친구에게 3개의 롤을 나누어 주었다"고 했으므로, 롤이 구분되지 않더라도, 각 롤이 어떤 친구에게 갈지가 무작위로 결정된다고 생각할 수 있습니다.

하지만, 확률을 계산할 때는 각 롤이 구분되지 않더라도, 가능한 모든 경우의 수를 고려해야 하므로, 실제로는 9개의 롤을 3개씩 3그룹으로 나누는 경우의 수를 구하

# Tokenize & Label 준비
- user의 instruction에는 label에 -100 처리를 통해 최적화에서 제외
- assistant turn만 학습하기 위함

In [10]:
from transformers import PreTrainedTokenizer

def _prepare_inputs(conversation, tokenizer):
    input_ids = []
    labels = []
    think_token_id = tokenizer.convert_tokens_to_ids("<think>")

    # For multi-turn conversations
    for conv in conversation[:-2]:
        _input_ids = tokenizer.apply_chat_template(
            [conv],
            tokenize=True
        )
        if conv['role'] == 'user':
            _labels = [-100] * len(_input_ids)
        elif conv['role'] == 'assistant':
            _labels = [-100] * 4 + _input_ids[4:-1] + [-100]
        
        input_ids += _input_ids
        labels += _labels

    _input_ids = tokenizer.apply_chat_template(
        conversation[-2:],
        tokenize=True
    )

    think_token_index = _input_ids.index(think_token_id) + 2
    input_ids += _input_ids
    labels += [-100]*think_token_index + _input_ids[think_token_index:]

    return input_ids, labels


def prepare_inputs(examples, tokenizer):
    input_ids_list = []
    labels_list = []

    for conversation in examples['conversation']:
        input_ids, labels = _prepare_inputs(conversation, tokenizer)
        input_ids_list.append(input_ids)
        labels_list.append(labels)
    return{
        "input_ids": input_ids_list,
        "labels": labels_list
    }

dataset = dataset.map(prepare_inputs, batched=True, fn_kwargs={'tokenizer': tokenizer})

In [11]:
# check tokenized data sample
print(tokenizer.decode(dataset[3]['input_ids']))

<|BOT|>user
Was the Hafele-Keating experiment, involving atomic clocks on airplanes, conducted in 1971?A: Pound-Rebka experiment, 1959
B: Miller experiment, 1925
C: Michelson-Morley experiment, 1887
D: Hafele-Keating experiment, 1971<|EOT|>
<|BOT|>assistant
<think>
Okay, the user is asking if the Hafele-Keating experiment with atomic clocks on airplanes happened in 1971. They've given multiple choices: A is Pound-Rebka in 1959, B is Miller in 1925, C is Michelson-Morley in 1887, and D is Hafele-Keating in 1971. 
Hmm, this seems straightforward. I remember the Hafele-Keating experiment was indeed in 1971. It was a famous test of Einstein's relativity where they flew atomic clocks on commercial jets around the world. The results showed time dilation effects-clocks moving eastward ran slower than stationary ones, westward ones ran faster, matching relativity predictions. 

Let me double-check the other options to be thorough. Pound-Rebka (1959) was about gravitational redshift using gamma

In [12]:
# comment: reasoning 종료 시점을 원활히 학습하기 위해 </think>은 label에서 제외하지 않았습니다.

sample = dataset[3]

print(f"{'Token':<15}{'input_ids':<15}{'label':<10}")
print("-"*35)
for ids, label in zip(sample['input_ids'], sample['labels']):
    token = tokenizer.decode(ids)
    print(f"{token.replace("\n", "\\n"):<15}{ids:<15}{label:<10}")

Token          input_ids      label     
-----------------------------------
<|BOT|>        125039         -100      
user           9604           -100      
\n             183            -100      
Was            108740         -100      
 the           263            -100      
 Haf           118062         -100      
ele            2668           -100      
-K             21350          -100      
eating         51084          -100      
 experiment    5194           -100      
,              11             -100      
 involving     3329           -100      
 atomic        27012          -100      
 clocks        34443          -100      
 on            360            -100      
 airplanes     55500          -100      
,              11             -100      
 conducted     9073           -100      
 in            268            -100      
               205            -100      
1              16             -100      
9              24             -100      
7              22    

In [13]:
from dataclasses import dataclass
from torch.utils.data import DataLoader

K=1024
@dataclass
class DataCollatorForSFT:
    tokenizer: PreTrainedTokenizer

    def __call__(self, instances):
        input_ids = [instance["input_ids"][:20*K] for instance in instances]
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )

        labels = [instance["labels"][:20*K] for instance in instances]
        labels = torch.nn.utils.rnn.pad_sequence(
            labels, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = labels.masked_fill(labels == self.tokenizer.pad_token_id, -100)

        return dict(
            input_ids=input_ids,
            labels=labels,
        )
    
collator = DataCollatorForSFT(tokenizer)

dataset.set_format('torch')
data_loader = DataLoader(
    dataset,
    collate_fn=collator,
    batch_size=4,
)

In [14]:
next(iter(data_loader))

{'input_ids': tensor([[125039,   9604,    183,  ..., 125032, 125032, 125032],
         [125039,   9604,    183,  ...,     13, 125040,    183],
         [125039,   9604,    183,  ..., 125032, 125032, 125032],
         [125039,   9604,    183,  ..., 125032, 125032, 125032]]),
 'labels': tensor([[  -100,   -100,   -100,  ...,   -100,   -100,   -100],
         [  -100,   -100,   -100,  ...,     13, 125040,    183],
         [  -100,   -100,   -100,  ...,   -100,   -100,   -100],
         [  -100,   -100,   -100,  ...,   -100,   -100,   -100]])}

# 모델 학습

In [15]:
# LoRA 설정
peft_config = LoraConfig(
    r=128,
    lora_alpha=256,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)

# 학습 인자 설정
training_args = SFTConfig(
    output_dir="./KORMo-sft-step-qlora-sft",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_bnb_8bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=5e-5,
    bf16=True,
    warmup_ratio=0.01,
    lr_scheduler_type="cosine",
    packing=True,
    report_to=None,
)

# SFT Trainer 초기화
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args,
)

Padding-free training is enabled, but the attention implementation is not set to a supported flash attention variant. Padding-free training flattens batches into a single sequence, and only the following implementations are known to reliably support this: flash_attention_2, flash_attention_3, kernels-community/flash-attn, kernels-community/flash-attn3, kernels-community/vllm-flash-attn3. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation` in the model configuration to one of these supported options or verify that your attention mechanism can handle flattened sequences.
You are using packing, but the attention implementation is not set to a supported flash attention variant. Packing gathers multiple samples into a single sequence, and only the following implementations are known to reliably support this: flash_attention_2, flash_attention_3, kernels-community/flash-attn, kernels-community/flash-attn3, kernels-community/vllm-fla

Packing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# 학습 시작
print("✓ 학습 시작...")
trainer.train()

# 모델 저장
print("\n✓ 학습 완료! 모델 저장 중...")
trainer.model.save_pretrained(training_args.output_dir)
tokenizer.save_pretrained(training_args.output_dir)
print(f"✓ 모델 저장 완료: {training_args.output_dir}")

✓ 학습 시작...


/raid/MLP/mjkim/KORMo-tutorial/.venv_kormo/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.658100
20,0.808900
30,0.566200
40,0.658600
50,0.573200
60,0.689200
70,0.674900
80,0.584000
90,0.518000
100,0.568800



✓ 학습 완료! 모델 저장 중...
✓ 모델 저장 완료: ./KORMo-IFT-step-qlora-sft


In [23]:
model = AutoModelForCausalLM.from_pretrained(
    training_args.output_dir,
    dtype='auto',
    trust_remote_code=True
).to('cuda')

# Non-think 추론
print("[Non-think 추론]")
messages = [{"role": "user", "content": test_prompt}]
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"입력: {test_prompt}")
print(f"출력: {response}")

# Non-think 추론
print("\n[Think 추론]")
messages = [{"role": "user", "content": test_prompt}]
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=True)
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"입력: {test_prompt}")
print(f"출력: {response}")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

[Non-think 추론]
입력: 영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
출력: user
영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
assistant
<think>

</think>
영희는 처음에 **연필 12개**를 가지고 있었습니다.
철수가 **절반**을 가져갔으므로,  
철수가 가져간 연필 수 = \( \frac{12}{2} = 6 \)개

따라서 영희에게 남은 연필 수 = \( 12 - 6 = 6 \)개

영수가 **공책 3개**를 가져갔다는 정보는 **연필과 관련이 없습니다**.  
공책은 연필이 아니므로, 영희의 연필 수에는 영향을 주지 않습니다.

### 따라서, 영희에게 남은 연필의 개수는  
\[
\boxed{6}
\]

[Think 추론]
입력: 영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
출력: user
영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
assistant
<think>
영희가 처음에 가지고 있던 연필의 수는 12개입니다. 철수가 이 연필의 절반을 가져갔으므로, 철수가 가져간 연필의 수는 12개의 절반인 6개입니다. 따라서, 철수가 가져간 후 영희에게 남은 연필의 수는 12개에서 6개를 뺀 6개입니다.

다음으로, 영수가 공책 3개를 가져갔다는 정보는 연필의 수와 관련이 없으므로, 이 부분은 문제의 해결에 영향을 미치지 않습니다.

결론적으로, 철수가 절반을 가져간 후 영희에게 남은 연필의 수는 6개입니다.
</think>

영희가 처음에 가지고 있던 연필은 12개입니다. 철수가 이 연필의 절반을 가져갔으므로, 철수가 가져간 연필의 수는 다음과 같습니다:

$$
\frac{12}{